# Part I. ETL Pipeline for Pre-Processing the Files

#### <font color='blue'>1.1 Import Python Package</font> 

In [3]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### <font color='blue'>1.2 Creating list of filepaths to process original event csv data files</font> 

In [5]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

D:\Dropbox (Personal)\Udacity\Data Engineering with AWS\project\cassandra


#### <font color='blue'>1.3 Processing the files to create the data file csv that will be used for Apache Casssandra tables</font> 

In [7]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [8]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Apache Cassandra

The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId
  
The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="image_event_datafile_new.jpg">

#### <font color='blue'>2.1 Creating a Cluster</font> 

In [12]:
# Make a connection to a Cassandra instance in local machine 
from cassandra.cluster import Cluster
cluster = Cluster(["localhost"], port=9042)

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### <font color='blue'>2.2 Create Keyspace</font> 

In [14]:
# Create a Keyspace
session.execute(
    """
    CREATE KEYSPACE IF NOT EXISTS cassandra
    WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }
    """
);

#### <font color='blue'>2.3 Set Keyspace</font> 

In [16]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('cassandra')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. With Apache Cassandra, we model the database tables on the queries we want to run.

#### Create queries to ask the following three questions of the data

##### <font color='red'> 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4 </font> 


##### <font color='red'> 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182 </font> 
    

##### <font color='red'> 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own' </font> 

#### <font color='blue'>2.4 Query Execution</font> 

#### <font color='blue'>Query 1: Give me the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4</font> 

The table will contain those columns:
- **sessionId, itemInSession** as *partition keys* for filtering and to gurantee uniqueness.
- *clustering columns*
  - **artist**
  - **song_name**
  - **song_length**

In [21]:
#Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query1 = "SELECT artist, song_title, song_length FROM songs_by_session WHERE session_id = 338 AND item_in_session = 4"

In [22]:
#Create table for query 1
session.execute(
    """
    DROP TABLE IF EXISTS songs_by_user_session
    """
);
session.execute(
    """
    CREATE TABLE IF NOT EXISTS songs_by_session
    (session_id int, item_in_session int, artist varchar, song_title varchar,
    song_length float,  PRIMARY KEY(session_id, item_in_session))
    ;"""
);

In [23]:
#check column indexes for insertion
file = 'event_datafile_new.csv'
df = pd.read_csv(file)
import pandas as pd
for i, c in enumerate(pd.read_csv(file).columns):
    print(c, i)

artist 0
firstName 1
gender 2
itemInSession 3
lastName 4
length 5
level 6
location 7
sessionId 8
song 9
userId 10


In [24]:
# Insert into the Apache Cassandra 
with open(file, encoding="utf8") as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        query = "INSERT INTO songs_by_session (session_id, item_in_session, artist, song_title, song_length) "
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        session.execute(
            query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5]))
        )

##### **Do a SELECT to verify that the data have been inserted into each table**

In [26]:
rows = session.execute(query1)
for x in rows:
    print(x)

Row(artist='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


##### **Verify from the dataframe**

In [28]:
#verify
df[(df.sessionId == 338) & (df.itemInSession == 4)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
964,Faithless,Ava,F,4,Robinson,495.3073,free,"New Haven-Milford, CT",338,Music Matters (Mark Knight Dub),50


#### <font color='blue'>Query 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182</font> 

The table will contain those columns:
- **userID, sessionID** and also **itemInSession** as *composite partition keys* for sorting and to guarantee uniqueness.
- *clustering columns*
  - **artist**
  - **song_name**
  - **first_name**
  - **last_name**

In [31]:
#Query 2:  Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
query2 = "SELECT artist, song_title, first_name, last_name FROM users_by_session WHERE userid = 10 AND sessionid = 182"

In [32]:
#Create table for query 2
session.execute(
    """
    DROP TABLE IF EXISTS users_by_session
    """
);
session.execute(
    """
    CREATE TABLE IF NOT EXISTS users_by_session
    (userid int, sessionid int, item_in_session int, artist varchar, song_title varchar, first_name varchar, last_name varchar,
    PRIMARY KEY ((userid, sessionid), item_in_session))
    """
);

In [33]:
with open(file, encoding="utf8") as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        query = "INSERT INTO users_by_session (userid, sessionid, item_in_session, artist, song_title, first_name, last_name) "
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(
            query,
            (
                int(line[10]),
                int(line[8]),
                int(line[3]),
                line[0],
                line[9],
                line[1],
                line[4],
            ),
        )

##### **Do a SELECT to verify that the data have been inserted into each table**

In [35]:
rows = session.execute(query2)
for x in rows:
    print(x)

Row(artist='Down To The Bone', song_title="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song_title='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song_title='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Lonnie Gordon', song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')


##### **Verify from the dataframe**

In [37]:
#verify
df[(df.sessionId == 182) & (df.userId == 10)]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
151,Down To The Bone,Sylvie,F,0,Cruz,333.76608,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Keep On Keepin' On,10
152,Three Drives,Sylvie,F,1,Cruz,411.63710,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Greece 2000,10
153,Sebastien Tellier,Sylvie,F,2,Cruz,377.73016,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Kilometer,10
154,Lonnie Gordon,Sylvie,F,3,Cruz,181.21098,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",182,Catch You Baby (Steve Pitron & Max Sanna Radio...,10


#### <font color='blue'>Query 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own' </font> 

The table will contain those columns:
- **song_name, userID** are *partition keys* to filter by.
- *clustering columns*
  - **first_name**
  - **last_name**

In [40]:
#Query 3:  Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own' 
query3 = "SELECT first_name, last_name FROM users_by_song WHERE song_name = 'All Hands Against His Own'"

In [41]:
#Create table for query 3
session.execute(
    """
    DROP TABLE IF EXISTS users_by_song
    """
);
session.execute(
    """
    CREATE TABLE IF NOT EXISTS users_by_song
    (song_name varchar, user_id int, first_name varchar, last_name varchar, PRIMARY KEY(song_name, user_id))
    """
);

In [42]:
with open(file, encoding="utf8") as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    for line in csvreader:
        query = "INSERT INTO users_by_song (song_name, user_id, first_name, last_name) "
        query = query + " VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]) ,line[1], line[4]))

##### **Verify from the dataframe**

In [44]:
rows = session.execute(query3)
for x in rows:
    print(x)

Row(first_name='Jacqueline', last_name='Lynch')
Row(first_name='Tegan', last_name='Levine')
Row(first_name='Sara', last_name='Johnson')


In [45]:
#verify
df[(df.song == 'All Hands Against His Own')]

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
219,The Black Keys,Sara,F,31,Johnson,196.91057,paid,"Winston-Salem, NC",152,All Hands Against His Own,95
2442,The Black Keys,Jacqueline,F,50,Lynch,196.91057,paid,"Atlanta-Sandy Springs-Roswell, GA",559,All Hands Against His Own,29
2646,The Black Keys,Tegan,F,25,Levine,196.91057,paid,"Portland-South Portland, ME",611,All Hands Against His Own,80


#### <font color='blue'>2.6 Drop the tables before closing out the sessions</font> 

In [47]:
session.execute(
    """
    DROP TABLE IF EXISTS songs_by_session
    """
);
session.execute(
    """
    DROP TABLE IF EXISTS users_by_session
    """
);
session.execute(
    """
    DROP TABLE IF EXISTS users_by_song
    """
);

#### <font color='blue'>2.7 Close the session and cluster connection¶</font> 

In [49]:
session.shutdown()
cluster.shutdown()